* Section 1

  - <a href="#ScriptSpt1pt1">Script S.1.1: Compute Bloch Sphere Angles</a>

  - <a href="#ScriptSpt1pt2">Script S.1.2: Rotation Matrices Around Z And Y Axes</a>

  - <a href="#ScriptSpt1pt3">Script S.1.3: Constructing The Multiplexor Matrix</a>

  - <a href="#ScriptSpt1pt4">Script S.1.4: Recursive Quantum Multiplexor Transformation</a>

  - <a href="#ScriptSpt1pt5">Script S.1.5: State Preparation For A 3-Qubit System</a>

* Section 2

  - <a href="#ScriptSpt2pt1">Script S.2.1: Imports And Setup For Givens Decomposition</a>

  - <a href="#ScriptSpt2pt2">Script S.2.2: Single Givens Rotation Constructor</a>

  - <a href="#ScriptSpt2pt3">Script S.2.3: Sequential Application Of Givens Rotations</a>

  - <a href="#ScriptSpt2pt4">Script S.2.4: Decomposition Of A Random [4$ \times $4] Unitary</a>

* Section 3

  - <a href="#ScriptSpt3pt1">Script S.3.1: MSB-First Bits And Gray Code</a>

  - <a href="#ScriptSpt3pt2">Script S.3.2: Walsh Coefficients Via FWHT</a>

  - <a href="#ScriptSpt3pt3">Script S.3.3: Walsh Gate List (Gray-Ordered)</a>

  - <a href="#ScriptSpt3pt4">Script S.3.4: Build And Peephole-Optimize Circuit</a>

  - <a href="#ScriptSpt3pt5">Script S.3.5: Test: Walsh Synthesis Of A Diagonal Unitary</a>

# Installation of QFlux

In [ ]:
!pip install qflux

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━

# Section 1

## Script S.1.1: Compute Bloch Sphere Angles <a name="ScriptSpt1pt1"></a>

In [ ]:
import numpy as np
def compute_bloch_angles(c0, c1):
    norm = np.sqrt(np.abs(c0)**2 + np.abs(c1)**2)
    alpha, beta = c0/norm, c1/norm
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta * np.conjugate(alpha) / (np.abs(beta)*np.abs(alpha)))
    return theta, phi


## Script S.1.2: Rotation Matrices Around Z And Y Axes <a name="ScriptSpt1pt2"></a>

In [ ]:
def rz_matrix(phi):
    return np.array([[np.exp(-1j*phi/2), 0],
                     [0, np.exp(1j*phi/2)]])

def ry_matrix(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2),  np.cos(theta/2)]])


## Script S.1.3: Constructing The Multiplexor Matrix <a name="ScriptSpt1pt3"></a>

In [ ]:
from scipy.linalg import block_diag

def multiplexor_matrix(n, vector, bit=0):
    bit = int(bool(bit))
    multiplexor = None
    for i in np.arange(0,2**n,2):
        c0, c1 = vector[i], vector[i+1]
        theta, phi = compute_bloch_angles(c0, c1)
        r = ry_matrix(bit*np.pi - theta) @ rz_matrix(-phi)
        multiplexor = block_diag(multiplexor, r) if multiplexor is not None else r
    return multiplexor


## Script S.1.4: Recursive Quantum Multiplexor Transformation <a name="ScriptSpt1pt4"></a>

In [ ]:
def rotate_to_vacuum_matrix(vector_input):
    ndim = vector_input.size
    if ndim & (ndim - 1) != 0:
        raise ValueError("Dimension must be a power of 2.")
    n = int(np.log2(ndim))
    total_matrix = np.eye(ndim)
    vector_k = vector_input.copy()
    for k in np.arange(n,0,-1):
        vector_pruned = vector_k if k==n else vector_new[::2]
        multiplexor = multiplexor_matrix(k, vector_pruned)
        multiplexor_padded = multiplexor if k==n else np.kron(multiplexor, np.eye(2*(n-k)))
        total_matrix = multiplexor_padded @ total_matrix
        vector_k = multiplexor_padded @ vector_k
        vector_new = multiplexor @ vector_pruned
    total_matrix *= np.conjugate(vector_k[0])
    return total_matrix


## Script S.1.5: State Preparation For A 3-Qubit System <a name="ScriptSpt1pt5"></a>

In [ ]:
from numpy import linalg as LA
np.random.seed(42)
nq = 3
state_vector = (2*np.random.rand(8)-1) * np.exp(1j*2*np.pi*np.random.rand(8))
state_vector /= LA.norm(state_vector)
mrot = rotate_to_vacuum_matrix(state_vector)
rot_vector = mrot.dot(state_vector)
back_vector = np.conjugate(mrot.T).dot(rot_vector)



# Section 2

## Script S.2.1: Imports And Setup For Givens Decomposition <a name="ScriptSpt2pt1"></a>

In [ ]:
!pip install graycode
import numpy as np
import graycode


## Script S.2.2: Single Givens Rotation Constructor <a name="ScriptSpt2pt2"></a>

In [ ]:
def GivensRotation(i, j, k, A):
    """
    Construct a two-level unitary that zeros out A[j,i] by rotating rows j and k.
    Indices i, j, k are zero-based.
    """
    ndim = A.shape[0]
    G = np.eye(ndim, dtype=A.dtype)
    aji = A[j, i]
    if aji == 0:
        return G
    aki = A[k, i]
    norm = np.sqrt(abs(aji)**2 + abs(aki)**2)
    G[k, k] = np.conj(aki) / norm
    G[j, j] = aki / norm
    G[k, j] = np.conj(aji) / norm
    G[j, k] = -aji / norm
    return G


## Script S.2.3: Sequential Application Of Givens Rotations <a name="ScriptSpt2pt3"></a>

In [ ]:
def Gmatrix(U, gray=False, print_sequence=False):
    """
    Apply a full sequence of Givens rotations to triangularize U.
    Returns R @ Uphase with a det-phase adjustment.
    """
    ndim = U.shape[0]
    n = int(np.log2(ndim))
    R = np.eye(ndim, dtype=U.dtype)

    # Global phase adjustment so the final triangular form is the identity.
    detU = np.linalg.det(U)
    phase = np.exp(-1j * np.angle(detU) / ndim)
    Uphase = phase * np.eye(ndim, dtype=U.dtype)
    Um = Uphase @ U

    # Perform eliminations column by column
    if print_sequence:
        print("U = D", end=' ')
    gray_list = list(graycode.gen_gray_codes(n)) if gray else None
    for i in range(ndim - 1):
        for j in range(ndim - 1, i, -1):
            if gray:
                ii = int(bin(gray_list[i]), 2)
                jj = int(bin(gray_list[j]), 2)
                kk = int(bin(gray_list[j-1]), 2)
            else:
                ii, jj, kk = i, j, j-1
            if print_sequence:
                print(f"G^+[{ii+1},{jj+1},{kk+1}]", end=' ')
            G = GivensRotation(ii, jj, kk, Um)
            R = G @ R
            Um = G @ Um

    if print_sequence:
        print("\nD = (", np.conjugate(phase), ")*I")

    return R @ Uphase


## Script S.2.4: Decomposition Of A Random [4$ \times $4] Unitary <a name="ScriptSpt2pt4"></a>

In [ ]:
from scipy.stats import unitary_group
import numpy as np

def format_complex(z):
    return f"{z.real: .4f}{z.imag:+.4f}j"

def print_nicely(M):
    for row in M:
        print(" ".join(f"{format_complex(x):>16s}" for x in row))

# Reproducible random unitary
rng = np.random.default_rng(43)
ndim = 4
U = unitary_group.rvs(ndim, random_state=rng)

print("\nTarget unitary matrix U:")
print("------------------------")
print_nicely(U)

# Decomposition with Gray ordering
print("\nDecomposition:")
print("--------------")
RU = Gmatrix(U, gray=True, print_sequence=True)

U_transformed = RU @ U

print("\nTransformed matrix R @ U (should be identity):")
print("------------------------------------------------")
print_nicely(U_transformed)

# Numerical check
err = np.linalg.norm(U_transformed - np.eye(ndim))
print(f"\n||R @ U - I||_F = {err:.3e}")
assert np.allclose(U_transformed, np.eye(ndim), atol=1e-10)



Target unitary matrix U:
------------------------
  0.0705-0.1778j   0.2506+0.1192j  -0.6150-0.3046j  -0.4727-0.4382j
 -0.5747+0.4436j   0.3205+0.2315j  -0.3111+0.1705j   0.4100-0.1504j
 -0.2261-0.5996j   0.1046-0.1180j   0.2487+0.4313j   0.1247-0.5487j
 -0.1604+0.0027j  -0.2503-0.8241j  -0.3131-0.2393j   0.2670-0.0766j

Decomposition:
--------------
U = D G^+[1,3,4] G^+[1,4,2] G^+[1,2,1] G^+[2,3,4] G^+[2,4,2] G^+[4,3,4] 
D = ( (0.9928811675497028-0.11910913955334813j) )*I

Transformed matrix R @ U (should be identity):
------------------------------------------------
  1.0000+0.0000j   0.0000-0.0000j   0.0000+0.0000j   0.0000-0.0000j
  0.0000-0.0000j   1.0000+0.0000j   0.0000+0.0000j  -0.0000+0.0000j
  0.0000-0.0000j   0.0000+0.0000j   1.0000-0.0000j  -0.0000-0.0000j
 -0.0000+0.0000j   0.0000+0.0000j   0.0000-0.0000j   1.0000-0.0000j

||R @ U - I||_F = 7.590e-16


# Section 3

## Script S.3.1: Msb-First Bits And Gray Code <a name="ScriptSpt3pt1"></a>

In [ ]:
import numpy as np

def bits_msb(x, n):
    """MSB-first bit vector of length n."""
    return np.fromiter(f"{x:0{n}b}", dtype='U1').astype(int)

def gray_index(i):
    """Integer Gray code index: i ^ (i >> 1)."""
    return i ^ (i >> 1)

def msb_pos(bits):
    """Index (0..n-1, MSB-first) of highest-order 1, or -1 if all zero."""
    idx = np.where(bits == 1)[0]
    return -1 if idx.size == 0 else idx[-1]


## Script S.3.2: Walsh Coefficients Via FWHT <a name="ScriptSpt3pt2"></a>

In [ ]:
def fwht_inplace(a):
    """In-place Fast Walsh–Hadamard Transform (Hadamard ordering)."""
    h = 1
    n = a.shape[0]
    while h < n:
        for i in range(0, n, h << 1):
            j_end = i + h
            for j in range(i, j_end):
                x = a[j]
                y = a[j + h]
                a[j]     = x + y
                a[j + h] = x - y
        h <<= 1
    return a

def walsh_coefficients_from_phases(f):
    """
    Given real phases f_k (length 2**n), return a_j per Eq. (\\ref{eq:walsh_coef}).
    """
    f = np.asarray(f, dtype=float).copy()
    a = fwht_inplace(f) / f.size
    return a

def phases_from_diagonal(U_diag):
    """
    Accept a 1D array of diagonal entries of U (|U_kk|=1) and return principal phases f_k.
    """
    U_diag = np.asarray(U_diag, dtype=complex)
    return np.angle(U_diag)  # principal branch (-pi, pi]


## Script S.3.3: Walsh Gate List (Gray-Ordered) <a name="ScriptSpt3pt3"></a>

In [ ]:
def walsh_gate_list(a, n_qubits, eps=1e-9):
    """
    Produce a Gray-ordered list of ('C', c, t) CNOTs and ('R', q, theta) Rz for U=exp(i sum a_j w_j).
    Qubit indices use MSB-first order: q=0 is the most significant qubit.
    """
    assert len(a) == 2**n_qubits
    gates = []

    # Iterate Gray order over j=1..2^n-1 (the j=0 term is a global phase Rz on no qubit)
    for i in range(1, 2**n_qubits):
        j = gray_index(i)
        aj = a[j]
        if abs(aj) < eps:
            continue
        bits = bits_msb(j, n_qubits)           # MSB-first
        t = msb_pos(bits)                      # target = highest one bit
        if t < 0:                              # shouldn't happen for j>=1
            continue

        # forward parity ladder onto target
        for c in range(0, t):
            if bits[c] == 1:
                gates.append(('C', c, t))

        # the single-qubit Z rotation: e^{i a Z} = Rz(-2 a)
        gates.append(('R', t, -2.0 * aj))

        # uncompute parity
        for c in range(t-1, -1, -1):
            if bits[c] == 1:
                gates.append(('C', c, t))

    return gates


## Script S.3.4: Build And Peephole-Optimize Circuit <a name="ScriptSpt3pt4"></a>

In [ ]:
from qiskit import QuantumRegister, QuantumCircuit

def cx_cancel(gates):
    """Cancel adjacent identical CNOTs."""
    out = []
    for g in gates:
        if out and g[0]=='C' and g == out[-1]:
            out.pop()
        else:
            out.append(g)
    return out

def build_qiskit(gates, n_qubits):
    qr = QuantumRegister(n_qubits, 'q')
    qc = QuantumCircuit(qr)
    for tag,*args in gates:
        if tag == 'C':
            c, t = args
            qc.cx(c, t)
        elif tag == 'R':
            q, theta = args
            qc.rz(theta, q)
        else:
            raise ValueError(f"Unknown gate {tag}")
    return qc


## Script S.3.5: Test: Walsh Synthesis Of A Diagonal Unitary <a name="ScriptSpt3pt5"></a>

In [ ]:
# Example: 3-qubit diagonal unitary
n = 3
rng = np.random.default_rng(7)
f = rng.uniform(-np.pi, np.pi, size=2**n)  # phases f_k
a = walsh_coefficients_from_phases(f)      # a_j via FWHT

# Gate list -> small peephole optimization -> circuit
gates = walsh_gate_list(a, n)
gates_opt = cx_cancel(gates)
qc = build_qiskit(gates_opt, n)

print("Walsh coefficients a_j:", np.round(a, 6))
print("\nOptimized circuit:")
print(qc)
print("\nCircuit depth:", qc.depth(), "   # quick proxy for entangling depth")


Walsh coefficients a_j: [ 0.41109  -0.87257   0.68223  -0.455547  0.410754  1.309817  0.136809
 -0.836585]

Optimized circuit:
                                                                »
q_0: ────────────────────────────────────────────────────────■──»
                                            ┌─────────────┐┌─┴─┐»
q_1: ────────────────■───────────────────■──┤ Rz(-1.3645) ├┤ X ├»
     ┌────────────┐┌─┴─┐┌─────────────┐┌─┴─┐└─────────────┘└───┘»
q_2: ┤ Rz(1.7451) ├┤ X ├┤ Rz(0.91109) ├┤ X ├────────────────────»
     └────────────┘└───┘└─────────────┘└───┘                    »
«                                                                           »
«q_0: ──────────────────■────■───────────────────────────────────────────■──»
«     ┌──────────────┐┌─┴─┐  │                                           │  »
«q_1: ┤ Rz(-0.27362) ├┤ X ├──┼────■──────────────────■───────────────────┼──»
«     └──────────────┘└───┘┌─┴─┐┌─┴─┐┌────────────┐┌─┴─┐┌─────────────┐┌─┴─┐»
«q_2: ───────────────